Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (504090, 28, 28) (504090,)
Validation set (25000, 28, 28) (25000,)
Test set (18720, 28, 28) (18720,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (504090, 28, 28, 1) (504090, 10)
Validation set (25000, 28, 28, 1) (25000, 10)
Test set (18720, 28, 28, 1) (18720, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.517112
Minibatch accuracy: 12.5%
Validation accuracy: 10.8%
Minibatch loss at step 50: 1.901366
Minibatch accuracy: 50.0%
Validation accuracy: 51.5%
Minibatch loss at step 100: 0.960779
Minibatch accuracy: 68.8%
Validation accuracy: 70.1%
Minibatch loss at step 150: 1.263199
Minibatch accuracy: 62.5%
Validation accuracy: 75.4%
Minibatch loss at step 200: 0.801022
Minibatch accuracy: 68.8%
Validation accuracy: 76.9%
Minibatch loss at step 250: 0.716393
Minibatch accuracy: 75.0%
Validation accuracy: 77.0%
Minibatch loss at step 300: 0.562906
Minibatch accuracy: 87.5%
Validation accuracy: 79.0%
Minibatch loss at step 350: 0.608726
Minibatch accuracy: 81.2%
Validation accuracy: 79.5%
Minibatch loss at step 400: 0.664071
Minibatch accuracy: 75.0%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.084007
Minibatch accuracy: 100.0%
Validation accuracy: 81.1%
Minibatch loss at step 500: 0.808882
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [12]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
num_steps = 1001

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    relu = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(relu, [1,2,2,1], [1,2,2,1], padding='SAME')
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    relu = tf.nn.relu(conv + layer2_biases)
    hidden = tf.nn.max_pool(relu, [1,2,2,1], [1,2,2,1], padding='SAME')
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.203771
Minibatch accuracy: 12.5%
Validation accuracy: 9.9%
Minibatch loss at step 50: 1.894837
Minibatch accuracy: 43.8%
Validation accuracy: 47.9%
Minibatch loss at step 100: 1.133459
Minibatch accuracy: 50.0%
Validation accuracy: 63.4%
Minibatch loss at step 150: 1.372210
Minibatch accuracy: 62.5%
Validation accuracy: 70.2%
Minibatch loss at step 200: 0.962181
Minibatch accuracy: 75.0%
Validation accuracy: 73.4%
Minibatch loss at step 250: 0.808819
Minibatch accuracy: 81.2%
Validation accuracy: 76.4%
Minibatch loss at step 300: 0.720418
Minibatch accuracy: 75.0%
Validation accuracy: 79.7%
Minibatch loss at step 350: 0.646657
Minibatch accuracy: 81.2%
Validation accuracy: 79.7%
Minibatch loss at step 400: 0.584711
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Minibatch loss at step 450: 0.102403
Minibatch accuracy: 100.0%
Validation accuracy: 81.6%
Minibatch loss at step 500: 0.752791
Minibatch accuracy: 68.8%
Validation accuracy: 80.2%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [49]:
for keep_rate in [0.95, 0.875, 0.75, 0.5, 0.25]:
    for initial_learning_rate in [0.8, 0.4, 0.2, 0.1, 0.05, 0.025, 0.0125, 0.00625, 0.003125]:
        for decay_rate_value in [0.6, 0.7, 0.8, 0.9, 0.95]:

            graph = tf.Graph()

            with graph.as_default():

              # Input data.
              tf_train_dataset = tf.placeholder(
                tf.float32, shape=(batch_size, image_size, image_size, num_channels))
              tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
              tf_valid_dataset = tf.constant(valid_dataset)
              tf_test_dataset = tf.constant(test_dataset)
              
              # Variables.
              layer1_weights = tf.Variable(tf.truncated_normal(
                  [patch_size, patch_size, num_channels, depth], stddev=0.1))
              layer1_biases = tf.Variable(tf.zeros([depth]))
              layer2_weights = tf.Variable(tf.truncated_normal(
                  [patch_size, patch_size, depth, depth], stddev=0.1))
              layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
              layer3_weights = tf.Variable(tf.truncated_normal(
                  [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
              layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
              layer4_weights = tf.Variable(tf.truncated_normal(
                  [num_hidden, num_labels], stddev=0.1))
              layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
              
              # Model.
              # try 1, 2 more layers
              def model(data):
                conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
                relu = tf.nn.relu(conv + layer1_biases)
                dropout = tf.nn.dropout(relu, keep_rate)
                hidden = tf.nn.max_pool(dropout, [1,2,2,1], [1,2,2,1], padding='SAME')
                conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
                relu = tf.nn.relu(conv + layer2_biases)
                dropout = tf.nn.dropout(relu, keep_rate)
                hidden = tf.nn.max_pool(dropout, [1,2,2,1], [1,2,2,1], padding='SAME')
                shape = hidden.get_shape().as_list()
                reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
                hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
                return tf.matmul(hidden, layer4_weights) + layer4_biases
              
              # Training computation.
              logits = model(tf_train_dataset)
              loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
                
              # Optimizer with learning rate decay
              global_step = tf.Variable(0)  # count the number of steps taken.
              decay_steps = 50
              decay_rate = decay_rate_value
              learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_steps, decay_rate)
              optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

              
              # Predictions for the training, validation, and test data.
              train_prediction = tf.nn.softmax(logits)
              valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
              test_prediction = tf.nn.softmax(model(tf_test_dataset))


            with tf.Session(graph=graph) as session:
              tf.initialize_all_variables().run()
              print('Initialized kr: {:g}, ilr: {:g}, drv: {:g}'.format(keep_rate, initial_learning_rate, decay_rate_value))
              for step in range(num_steps):
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
                _, l, predictions = session.run(
                  [optimizer, loss, train_prediction], feed_dict=feed_dict)
                # if (step % 50 == 0):
                #   print('Minibatch loss at step %d: %f' % (step, l))
                #   print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                #   print('Validation accuracy: %.1f%%' % accuracy(
                #     valid_prediction.eval(), valid_labels))
              print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

            

Initialized kr: 0.95, ilr: 0.8, drv: 0.6
Test accuracy: 10.0%
Initialized kr: 0.95, ilr: 0.8, drv: 0.7
Test accuracy: 10.0%
Initialized kr: 0.95, ilr: 0.8, drv: 0.8
Test accuracy: 10.0%
Initialized kr: 0.95, ilr: 0.8, drv: 0.9
Test accuracy: 10.0%
Initialized kr: 0.95, ilr: 0.8, drv: 0.95
Test accuracy: 10.0%
Initialized kr: 0.95, ilr: 0.4, drv: 0.6
Test accuracy: 10.0%
Initialized kr: 0.95, ilr: 0.4, drv: 0.7
Test accuracy: 10.0%
Initialized kr: 0.95, ilr: 0.4, drv: 0.8
Test accuracy: 10.0%
Initialized kr: 0.95, ilr: 0.4, drv: 0.9
Test accuracy: 29.3%
Initialized kr: 0.95, ilr: 0.4, drv: 0.95
Test accuracy: 10.0%
Initialized kr: 0.95, ilr: 0.2, drv: 0.6
Test accuracy: 86.9%
Initialized kr: 0.95, ilr: 0.2, drv: 0.7
Test accuracy: 85.7%
Initialized kr: 0.95, ilr: 0.2, drv: 0.8
Test accuracy: 89.1%
Initialized kr: 0.95, ilr: 0.2, drv: 0.9
Test accuracy: 91.0%
Initialized kr: 0.95, ilr: 0.2, drv: 0.95
Test accuracy: 91.2%
Initialized kr: 0.95, ilr: 0.1, drv: 0.6
Test accuracy: 86.7%
Initi

---
Problem 2 - initial grid search
---------

Best results above

__KR__ = keep rate for `dropout()`

__ILR__ = initial learning rate

__DR__ = learning decay rate percentage per decay steps (`decay_steps=50`)


| KR | ILR | DR | Accuracy         
| :- |:- | :- | :-:
| 0.95 | 0.2 | 0.9 | 91.0%
| 0.95 | 0.2 | 0.95 | 91.2%
| 0.95 | 0.1 | 0.9 | 90.4%
| 0.95 | 0.1 | 0.95 | 91.2%
| 0.875 | 0.1 | 0.9 | 90.5%
| 0.875 | 0.1 | 0.95 | 91.0%
| 0.875 | 0.05 | 0.95 | 90.1%
| 0.75 | 0.1 | 0.9 | 90.9%

---

In [50]:
for keep_rate in [1, 0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6]:
    for initial_learning_rate in [0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.05, 0.04, 0.03]:
        for decay_rate_value in [0.975, 0.95, 0.925, 0.9, 0.875, 0.85, 0.825]:

            graph = tf.Graph()

            with graph.as_default():

              # Input data.
              tf_train_dataset = tf.placeholder(
                tf.float32, shape=(batch_size, image_size, image_size, num_channels))
              tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
              tf_valid_dataset = tf.constant(valid_dataset)
              tf_test_dataset = tf.constant(test_dataset)
              
              # Variables.
              layer1_weights = tf.Variable(tf.truncated_normal(
                  [patch_size, patch_size, num_channels, depth], stddev=0.1))
              layer1_biases = tf.Variable(tf.zeros([depth]))
              layer2_weights = tf.Variable(tf.truncated_normal(
                  [patch_size, patch_size, depth, depth], stddev=0.1))
              layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
              layer3_weights = tf.Variable(tf.truncated_normal(
                  [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
              layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
              layer4_weights = tf.Variable(tf.truncated_normal(
                  [num_hidden, num_labels], stddev=0.1))
              layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
              
              # Model.
              # try 1, 2 more layers
              def model(data):
                conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
                relu = tf.nn.relu(conv + layer1_biases)
                dropout = tf.nn.dropout(relu, keep_rate)
                hidden = tf.nn.max_pool(dropout, [1,2,2,1], [1,2,2,1], padding='SAME')
                conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
                relu = tf.nn.relu(conv + layer2_biases)
                dropout = tf.nn.dropout(relu, keep_rate)
                hidden = tf.nn.max_pool(dropout, [1,2,2,1], [1,2,2,1], padding='SAME')
                shape = hidden.get_shape().as_list()
                reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
                hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
                return tf.matmul(hidden, layer4_weights) + layer4_biases
              
              # Training computation.
              logits = model(tf_train_dataset)
              loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
                
              # Optimizer with learning rate decay
              global_step = tf.Variable(0)  # count the number of steps taken.
              decay_steps = 50
              decay_rate = decay_rate_value
              learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_steps, decay_rate)
              optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

              
              # Predictions for the training, validation, and test data.
              train_prediction = tf.nn.softmax(logits)
              valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
              test_prediction = tf.nn.softmax(model(tf_test_dataset))


            with tf.Session(graph=graph) as session:
              tf.initialize_all_variables().run()
              print('Initialized kr: {:g}, ilr: {:g}, drv: {:g}'.format(keep_rate, initial_learning_rate, decay_rate_value))
              for step in range(num_steps):
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
                _, l, predictions = session.run(
                  [optimizer, loss, train_prediction], feed_dict=feed_dict)
                # if (step % 50 == 0):
                #   print('Minibatch loss at step %d: %f' % (step, l))
                #   print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                #   print('Validation accuracy: %.1f%%' % accuracy(
                #     valid_prediction.eval(), valid_labels))
              print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
            

Initialized kr: 1, ilr: 0.35, drv: 0.975
Test accuracy: 10.0%
Initialized kr: 1, ilr: 0.35, drv: 0.95
Test accuracy: 10.0%
Initialized kr: 1, ilr: 0.35, drv: 0.925
Test accuracy: 10.0%
Initialized kr: 1, ilr: 0.35, drv: 0.9
Test accuracy: 10.0%
Initialized kr: 1, ilr: 0.35, drv: 0.875
Test accuracy: 10.0%
Initialized kr: 1, ilr: 0.35, drv: 0.85
Test accuracy: 10.0%
Initialized kr: 1, ilr: 0.35, drv: 0.825
Test accuracy: 10.0%
Initialized kr: 1, ilr: 0.3, drv: 0.975
Test accuracy: 10.0%
Initialized kr: 1, ilr: 0.3, drv: 0.95
Test accuracy: 10.0%
Initialized kr: 1, ilr: 0.3, drv: 0.925
Test accuracy: 10.0%
Initialized kr: 1, ilr: 0.3, drv: 0.9
Test accuracy: 88.8%
Initialized kr: 1, ilr: 0.3, drv: 0.875
Test accuracy: 10.0%
Initialized kr: 1, ilr: 0.3, drv: 0.85
Test accuracy: 10.0%
Initialized kr: 1, ilr: 0.3, drv: 0.825
Test accuracy: 10.0%
Initialized kr: 1, ilr: 0.25, drv: 0.975
Test accuracy: 89.9%
Initialized kr: 1, ilr: 0.25, drv: 0.95
Test accuracy: 89.4%
Initialized kr: 1, ilr: 

---
Problem 2 - second grid search
---------

__KR__ = keep rate for `dropout()`

__ILR__ = initial learning rate

__DR__ = learning decay rate percentage per decay steps (`decay_steps=50`)

Best result


| KR | ILR | DR | Accuracy         
| :- |:- | :- | :-:
| 0.95 | 0.3 | 0.9 | 91.9%

Other good results at or over 91%

| KR | ILR | DR | Accuracy         
| :- |:- | :- | :-:
| 1 | 0.25 | 0.875 | 91.7%
| 1 | 0.25 | 0.85 | 91.3%
| 1 | 0.2 | 0.925 | 91.0%
| 1 | 0.2 | 0.875 | 91.5%
| 1 | 0.2 | 0.85 | 91.2%
| 1 | 0.15 | 0.975 | 91.5%
| 1 | 0.15 | 0.95 | 91.1%
| 1 | 0.15 | 0.9 | 91.1%
| 1 | 0.15 | 0.875 | 91.0%
| 0.95 | 0.25 | 0.95 | 91.4%
| 0.95 | 0.2 | 0.875 | 91.6%
| 0.95 | 0.15 | 0.95 | 91.2%
| 0.95 | 0.15 | 0.875 | 91.1%
| 0.95 | 0.1 | 0.95 | 91.0%
| 0.95 | 0.1 | 0.925 | 91.0%
| 0.9 | 0.15 | 0.975 | 91.0%
| 0.9 | 0.15 | 0.95 | 91.6%
| 0.9 | 0.15 | 0.925 | 91.3%
| 0.9 | 0.15 | 0.85 | 91.1%
| 0.9 | 0.1 | 0.95 | 91.2%
| 0.9 | 0.1 | 0.9 | 91.0%
| 0.85 | 0.2 | 0.9 | 91.2%
| 0.85 | 0.15 | 0.925 | 91.8%
| 0.85 | 0.1 | 0.95 | 91.0%
| 0.8 | 0.15 | 0.975 | 91.0%
| 0.8 | 0.1 | 0.95 | 91.0%
| 0.75 | 0.15 | 0.85 | 91.3%
| 0.75 | 0.1 | 0.925 | 91.4%


---